In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import psycopg2
import streamlit as st

In [2]:
#Connect to local postgres DB
conn = psycopg2.connect(**st.secrets["postgres"])
cur = conn.cursor()

In [3]:
def run_query(query_path):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

## Questions To Answer:

- What is the proportion between equity and fixed income over time?
- What is the proportion between industries over time?
- What is the proportion between regions over time?
- What are the equity inflows over time? Assessing ownership %
- What industries has the fund been a buyer in?
- What regions/countries has the fund been buying

### Equity and Fixed Income Proportions Over Time

In [4]:
equity_fi_df = run_query('SQL/eq_fi_proportions.sql')
equity_fi_df

,year,Equity Percentage,Fixed Income Percentage
0,1998,39.66,60.34
1,1999,41.87,58.13
2,2000,38.13,61.87
3,2001,40.78,59.22
4,2002,36.74,63.26
5,2003,42.91,57.09
6,2004,39.61,60.39
7,2005,46.47,53.53
8,2006,38.18,61.82
9,2007,45.81,54.19


### Industry Proportions Over Time

In [5]:
industry_prop_df = run_query('SQL/industry_proportions.sql')
industry_prop_df

,year,Industry,Proportion of Fund
0,1998,Basic Materials,1.67
1,1998,Consumer Discretionary,4.69
2,1998,Consumer Staples,4.68
3,1998,Corporate Bonds,1.28
4,1998,Energy,2.23
...,...,...,...
370,2022,Securitized Bonds,1.27
371,2022,Technology,12.19
372,2022,Telecommunications,2.13
373,2022,Treasuries,19.42


### Region Proportions Over Time

In [6]:
region_prop_df = run_query('SQL/region_proportions.sql')
region_prop_df

,year,region,proportion
0,1998,Asia,18.77
1,1998,Europe,50.76
2,1998,International,2.63
3,1998,Latin America,0.10
4,1998,Middle East,0.03
...,...,...,...
186,2022,International,0.73
187,2022,Latin America,0.99
188,2022,Middle East,0.44
189,2022,North America,46.06


### Ownership By Industry Over Time

In [7]:
industry_ownership_df = run_query('SQL/ownership_industry.sql')
industry_ownership_df

,year,industry,avg_percent_ownership
0,1998,Basic Materials,0.05
1,1998,Consumer Discretionary,0.05
2,1998,Consumer Staples,0.05
3,1998,Energy,0.04
4,1998,Financials,0.05
...,...,...,...
270,2022,Industrials,1.11
271,2022,Real Estate,1.33
272,2022,Technology,1.05
273,2022,Telecommunications,1.01


### Cumulative Change In Percent Ownership By Industry - Last 10 Years

In [8]:
ownership_change_industry_ten_df = run_query('SQL/cum_ownership_change_industry_ten_years.sql')
ownership_change_industry_ten_df

,industry,cumulative_percent_change_of_ownership
0,Real Estate,29.72
1,Telecommunications,16.72
2,Basic Materials,14.71
3,Industrials,14.62
4,Health Care,14.36
5,Technology,13.69
6,Financials,11.77
7,Consumer Discretionary,8.91
8,Utilities,4.41
9,Consumer Staples,3.12


### Cumulative Change In Percent Ownership By Region - Last 10 Years

In [9]:
ownership_change_region_ten_df = run_query('SQL/cum_ownership_change_region_ten_years.sql')
ownership_change_region_ten_df

,region,cumulative_percent_change_of_ownership
0,Oceania,54.58
1,Africa,47.72
2,Middle East,46.30
3,Latin America,25.38
4,Asia,19.83
5,North America,12.54
6,Europe,-9.81


### Dynamic Queries - Testing Zone

In [10]:
#Dynamic query function

def run_query_dynamic(query_path, num_years):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query, (num_years,))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

### Industry Dynamic

In [17]:
ownership_change_industry_dynamic_df = run_query_dynamic('SQL/dynamic/ownership_change_industry.sql',10 )
ownership_change_industry_dynamic_df

,industry,cumulative_percent_change_of_ownership
0,Real Estate,29.72
1,Telecommunications,16.72
2,Basic Materials,14.71
3,Industrials,14.62
4,Health Care,14.36
5,Technology,13.69
6,Financials,11.77
7,Consumer Discretionary,8.91
8,Utilities,4.41
9,Consumer Staples,3.12


### Country Dynamic

In [12]:
def run_query_dynamic_country(query_path, num_years, country):
    with open(query_path, 'r') as file:
        query = file.read()
    with conn.cursor() as cur:
        cur.execute(query, (num_years, country))
        results = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(results, columns=columns)

In [13]:
ownership_change_country_dynamic_df = run_query_dynamic_country('SQL/dynamic/ownership_change_country.sql',10,'United States')
ownership_change_country_dynamic_df

,country,cumulative_percent_change_of_ownership
0,United States,13.12
